Import some useful libraries.

In [ ]:
from datetime import UTC, datetime
from urllib.parse import urljoin

from lsst.rsp import RSPClient, get_access_token

Obtain the notebook token for the user.
We won't be using this token directly, since we'll instead use an `RSPClient`, but this demonstrates that the standard utility function is working and the user has a token.

In [ ]:
token = get_access_token()
assert token, "You have no notebook token"

Create an RSP client to talk to Gafaelfawr.
Right now, this requires hard-coding the Gafaelfawr API URL prefix.
This interface will change once we have service discovery.

In [ ]:
client = RSPClient("/")

Get the user's user information from their notebook token.

In [ ]:
r = await client.get("/auth/api/v1/user-info")
assert r.status_code == 200
user_info = r.json()

Print out information about the user.

In [ ]:
print("Your username is", user_info["username"])

# name and email are optional and may not be set for every user (if, for instance, the RSP uses GitHub
# authentication and the user doesn't release an email address or name).
if "name" in user_info:
    print("Your name is", user_info["name"])
if "email" in user_info:
    print("Your email address is", user_info["email"])

print("Your numeric UID is", user_info["uid"])
print("Your numeric GID is", user_info["gid"])
print("Your groups are", ", ".join(f"{g['name']} ({g['id']})" for g in user_info["groups"]))

Print out the user's quota information.

In [ ]:
if "quota" not in user_info:
    print("You have no quotas set")
else:
    quota = user_info["quota"]
    if "api" in quota:
        print("Your API quotas:")
        for service, amount in sorted(quota["api"].items()):
            print(f"  Service {service}: {amount} per minute")
    if "notebook" in quota:
        notebook = quota["notebook"]
        if not notebook["spawn"]:
            print("You may not create a notebook server")
        else:
            cpu = quota["notebook"]["cpu"]
            memory = quota["notebook"]["memory"]
            print(f"You may create a notebook server with up to {cpu} core equivalents and {memory}GiB of memory")
    if "tap" in quota:
        print("Your TAP quotas:")
        for service, rule in sorted(quota["tap"].items()):
            print(f"  Backend {service}: {rule['concurrent']} concurrent reqeusts")

Now, retrieve the metadata about the user's token specifically.

In [ ]:
r = await client.get("/auth/api/v1/token-info")
assert r.status_code == 200
token_info = r.json()

Display information about the user's token, and check that the token is not expired.

In [ ]:
print("Your username is", token_info["username"])
print("Your token identifier is", token_info["token"])
print("Your token type is", token_info["token_type"], "(will always be notebook when executing in Nublado)")
print("Your scopes are", ", ".join(token_info["scopes"]))
print("")

# Print out the times.
created = datetime.fromtimestamp(token_info["created"], tz=UTC)
expires = datetime.fromtimestamp(token_info["expires"], tz=UTC)
current = datetime.now(tz=UTC)
print("Your token was issued at:", created.isoformat(sep=" "))
print("Your token expires at:   ", expires.isoformat(sep=" "))
print("The time is currently:   ", current.isoformat(sep=" ", timespec="seconds"))

assert current >= created, "Your token was created after the current time?!"

assert current <= expires, f"Your token expired at {expires.isoformat(sep=' ')}"
print("Your token is VALID")